In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.gridspec import GridSpec

import pyarrow as pa
import pyarrow.parquet as pq

import seaborn as sns
pd.set_option('display.max_columns', None)

In [ ]:
ids = np.load('ids_seven_days_each_five_bimonth.npy')


patient = pd.read_parquet('Patients_lookup.parquet', engine='pyarrow')#dcarte.load('Patients','lookup',reload=False)
patient['participant_id'] = patient['id'].apply(lambda x:x[:5])
patient = patient[patient['participant_id'].isin(ids)]

dyads = pd.read_excel('dyads_unprotected.xlsx')

dyads['participant_id'] = dyads['Research ID'].apply(lambda x:x[:5])
dyads = dyads[dyads['participant_id'].isin(ids)]

demographics = pd.read_csv('redcap_16_nov_2023/Demographics_MinderHealthManageme_DATA_LABELS_2023-11-16_1127.csv')
demographics.columns.to_list()

In [ ]:
research = demographics[['Dyad number',
                         'Primary dementia diagnosis (research) (choice=Alzheimers Disease (typical - late onset))',
                         'Primary dementia diagnosis (research) (choice=Alzheimers Disease (atypical - early onset))',
                         'Primary dementia diagnosis (research) (choice=Alzheimers Disease (atypical - posterior cortical atrophy variant))',
                         'Primary dementia diagnosis (research) (choice=Alzheimers Disease (atypical - cortico-basal syndrome))',
                         'Primary dementia diagnosis (research) (choice=Alzheimers Disease (atypical - progressive aphasia variant))',
                         'Primary dementia diagnosis (research) (choice=Lewy Body Dementia)',
                         'Primary dementia diagnosis (research) (choice=Dementia in Parkinsons Disease)',
                         'Primary dementia diagnosis (research) (choice=Frontotemporal dementia (unspecified))',
                         'Primary dementia diagnosis (research) (choice=Frontotemporal dementia (behavioural variant/Picks disease))',
                         'Primary dementia diagnosis (research) (choice=Frontotemporal dementia (language variant/Primary Progressive Aphasia))',
                         'Primary dementia diagnosis (research) (choice=Vascular dementia)',
                         'Primary dementia diagnosis (research) (choice=Mild cognitive impairment (MCI))',
                         'Primary dementia diagnosis (research) (choice=Early-onset dementia)',
                         'Primary dementia diagnosis (research) (choice=Alcohol-related brain damage)',
                         'Primary dementia diagnosis (research) (choice=Normal Pressure Hydrocephalus)',
                         'Primary dementia diagnosis (research) (choice=Traumatic Brain Injury)',
                         'Primary dementia diagnosis (research) (choice=No dementia)',
                         'Primary dementia diagnosis (research) (choice=Stroke)',
                         'Primary dementia diagnosis (research) (choice=Subjective Cognitive Impairment)']]

research  = research.rename(columns={"Dyad number": "DYAD"})
research.head(3)

In [ ]:
alzheimers = ['Primary dementia diagnosis (research) (choice=Alzheimers Disease (typical - late onset))',
         'Primary dementia diagnosis (research) (choice=Alzheimers Disease (atypical - early onset))',
         'Primary dementia diagnosis (research) (choice=Alzheimers Disease (atypical - posterior cortical atrophy variant))',
         'Primary dementia diagnosis (research) (choice=Alzheimers Disease (atypical - cortico-basal syndrome))',
         'Primary dementia diagnosis (research) (choice=Alzheimers Disease (atypical - progressive aphasia variant))']
research['research alzheimers'] = research[alzheimers].ne('Unchecked').any(axis=1)
dementias =[ 'Primary dementia diagnosis (research) (choice=Lewy Body Dementia)',
             'Primary dementia diagnosis (research) (choice=Dementia in Parkinsons Disease)',
             'Primary dementia diagnosis (research) (choice=Frontotemporal dementia (unspecified))',
             'Primary dementia diagnosis (research) (choice=Frontotemporal dementia (behavioural variant/Picks disease))',
             'Primary dementia diagnosis (research) (choice=Frontotemporal dementia (language variant/Primary Progressive Aphasia))',
             'Primary dementia diagnosis (research) (choice=Vascular dementia)',
             'Primary dementia diagnosis (research) (choice=Early-onset dementia)']
research['research dementias'] = research[dementias].ne('Unchecked').any(axis=1)
research['research other'] = research[['Primary dementia diagnosis (research) (choice=Alcohol-related brain damage)',
                         'Primary dementia diagnosis (research) (choice=Normal Pressure Hydrocephalus)',
                         'Primary dementia diagnosis (research) (choice=Traumatic Brain Injury)',
                         'Primary dementia diagnosis (research) (choice=No dementia)',
                         'Primary dementia diagnosis (research) (choice=Stroke)']].ne('Unchecked').any(axis=1)
research['research mci'] = research[['Primary dementia diagnosis (research) (choice=Mild cognitive impairment (MCI))',
                                    'Primary dementia diagnosis (research) (choice=Subjective Cognitive Impairment)']].ne('Unchecked').any(axis=1)

merge_research = dyads.merge(research, on='DYAD')
merge_research['Research Diagnosis'] ='Unknown'
merge_research.loc[merge_research['research other'] ==True, 'Research Diagnosis'] = 'other'
merge_research.loc[merge_research['research alzheimers'] ==True, 'Research Diagnosis'] = 'Alzheimers'
merge_research.loc[merge_research['research dementias'] ==True, 'Research Diagnosis'] = 'Dementias'
merge_research.loc[merge_research['research mci'] ==True, 'Research Diagnosis'] = 'MCI'
merge_research.loc[(merge_research['research alzheimers'] ==True)&(merge_research['research dementias'] ==True), 'Research Diagnosis'] = 'Alzheimers and Dementias'

merge_research['research_alzheimer_diagnosis'] ='no'
merge_research.loc[(merge_research['Research Diagnosis'] =='Alzheimers and Dementias')|(merge_research['Research Diagnosis']=='Alzheimers'),'research_alzheimer_diagnosis'] = 'yes'
merge_research.loc[(merge_research['Research Diagnosis'] =='Unknown'),'research_alzheimer_diagnosis'] = 'no'

merge_research=merge_research[[ 'participant_id','research alzheimers','research dementias','research other','research mci','Research Diagnosis','research_alzheimer_diagnosis']]
merge_research

In [ ]:
merge_research[merge_research['Research Diagnosis']=='Alzheimers and Dementias']

In [ ]:
merge_research.to_excel("redcap_research_diagnosis_08_FEB_2024.xlsx")

In [ ]:
demographics = demographics[['Dyad number', 'Year of birth PwD',
 'Gender PwD','Clinical Diagnosis (choice=Alzheimers Disease (typical - late onset))',
 'Clinical Diagnosis (choice=Alzheimers Disease (atypical - early onset))',
 'Clinical Diagnosis (choice=Alzheimers Disease (atypical - posterior cortical atrophy variant))',
 'Clinical Diagnosis (choice=Alzheimers Disease (atypical - cortico-basal syndrome))',
 'Clinical Diagnosis (choice=Alzheimers Disease (atypical - progressive aphasia variant))',
 'Clinical Diagnosis (choice=Lewy Body Dementia)',
 'Clinical Diagnosis (choice=Dementia in Parkinsons Disease)',
 'Clinical Diagnosis (choice=Frontotemporal dementia (unspecified))',
 'Clinical Diagnosis (choice=Frontotemporal dementia (behavioural variant/Picks disease))',
 'Clinical Diagnosis (choice=Frontotemporal dementia (language variant/Primary Progressive Aphasia))',
 'Clinical Diagnosis (choice=Vascular dementia)',
 'Clinical Diagnosis (choice=Mild cognitive impairment (MCI))',
 'Clinical Diagnosis (choice=Young-onset dementia)',
 'Clinical Diagnosis (choice=Alcohol-related brain damage)',
 'Clinical Diagnosis (choice=Mixed Alzheimers and Vascular Dementia)',
 'Clinical Diagnosis (choice=Dementia in Alzheimer disease, atypical or mixed type)',
 'Clinical Diagnosis (choice=Normal Pressure Hydrocephalus)',
 'Clinical Diagnosis (choice=Traumatic Brain Injury)',
 'Clinical Diagnosis (choice=No dementia)',
 'Clinical Diagnosis (choice=Stroke)',
 'Clinical Diagnosis (choice=Frailty)']]
#diagnosis, imaging results and regions affected, comorbidities, smoke, drink, house style, mobility, pets
demographics  = demographics.rename(columns={"Dyad number": "DYAD"})
demographics.head(3)

In [ ]:
alzheimers = ['Clinical Diagnosis (choice=Alzheimers Disease (typical - late onset))',
             'Clinical Diagnosis (choice=Alzheimers Disease (atypical - early onset))',
            'Clinical Diagnosis (choice=Alzheimers Disease (atypical - posterior cortical atrophy variant))',
             'Clinical Diagnosis (choice=Alzheimers Disease (atypical - cortico-basal syndrome))',
              'Clinical Diagnosis (choice=Alzheimers Disease (atypical - progressive aphasia variant))']
demographics['alzheimers'] = demographics[alzheimers].ne('Unchecked').any(axis=1)

dementias =['Clinical Diagnosis (choice=Lewy Body Dementia)',
            'Clinical Diagnosis (choice=Dementia in Parkinsons Disease)',
            'Clinical Diagnosis (choice=Frontotemporal dementia (unspecified))', 
            'Clinical Diagnosis (choice=Frontotemporal dementia (behavioural variant/Picks disease))',
             'Clinical Diagnosis (choice=Frontotemporal dementia (language variant/Primary Progressive Aphasia))',
            'Clinical Diagnosis (choice=Vascular dementia)',
            'Clinical Diagnosis (choice=Young-onset dementia)']
demographics['dementias'] = demographics[dementias].ne('Unchecked').any(axis=1)

other =['Clinical Diagnosis (choice=Alcohol-related brain damage)',
         'Clinical Diagnosis (choice=Normal Pressure Hydrocephalus)',
         'Clinical Diagnosis (choice=Traumatic Brain Injury)',
         'Clinical Diagnosis (choice=No dementia)',
         'Clinical Diagnosis (choice=Stroke)',
         'Clinical Diagnosis (choice=Frailty)']
demographics['other'] = demographics[other].ne('Unchecked').any(axis=1)

demographics['alzheimer_and_dementia'] = demographics[['Clinical Diagnosis (choice=Mixed Alzheimers and Vascular Dementia)',
                                        'Clinical Diagnosis (choice=Dementia in Alzheimer disease, atypical or mixed type)']].ne('Unchecked').any(axis=1)
demographics['mci'] = demographics[['Clinical Diagnosis (choice=Mild cognitive impairment (MCI))']].ne('Unchecked').any(axis=1)

merge1 = dyads.merge(demographics, on='DYAD')
merge1 = merge1[['participant_id', 'Year of birth PwD',
       'Gender PwD',
       'Clinical Diagnosis (choice=Alzheimers Disease (typical - late onset))',
       'Clinical Diagnosis (choice=Alzheimers Disease (atypical - early onset))',
       'Clinical Diagnosis (choice=Alzheimers Disease (atypical - posterior cortical atrophy variant))',
       'Clinical Diagnosis (choice=Alzheimers Disease (atypical - cortico-basal syndrome))',
       'Clinical Diagnosis (choice=Alzheimers Disease (atypical - progressive aphasia variant))',
       'Clinical Diagnosis (choice=Lewy Body Dementia)',
       'Clinical Diagnosis (choice=Dementia in Parkinsons Disease)',
       'Clinical Diagnosis (choice=Frontotemporal dementia (unspecified))',
       'Clinical Diagnosis (choice=Frontotemporal dementia (behavioural variant/Picks disease))',
       'Clinical Diagnosis (choice=Frontotemporal dementia (language variant/Primary Progressive Aphasia))',
       'Clinical Diagnosis (choice=Vascular dementia)',
       'Clinical Diagnosis (choice=Mild cognitive impairment (MCI))',
       'Clinical Diagnosis (choice=Young-onset dementia)',
       'Clinical Diagnosis (choice=Alcohol-related brain damage)',
       'Clinical Diagnosis (choice=Mixed Alzheimers and Vascular Dementia)',
       'Clinical Diagnosis (choice=Dementia in Alzheimer disease, atypical or mixed type)',
       'Clinical Diagnosis (choice=Normal Pressure Hydrocephalus)',
       'Clinical Diagnosis (choice=Traumatic Brain Injury)',
       'Clinical Diagnosis (choice=No dementia)',
       'Clinical Diagnosis (choice=Stroke)',
       'Clinical Diagnosis (choice=Frailty)', 'alzheimers', 'dementias',
       'alzheimer_and_dementia', 'mci','other']]

merge1['Clinical Diagnosis'] ='Unknown'
merge1.loc[merge1['other'] ==True, 'Clinical Diagnosis'] = 'other'
merge1.loc[merge1['alzheimers'] ==True, 'Clinical Diagnosis'] = 'Alzheimers'
merge1.loc[merge1['dementias'] ==True, 'Clinical Diagnosis'] = 'Dementias'
merge1.loc[merge1['mci'] ==True, 'Clinical Diagnosis'] = 'MCI'
merge1.loc[(merge1['alzheimer_and_dementia'] ==True), 'Clinical Diagnosis'] = 'Alzheimers and Dementias'
merge1.loc[(merge1['alzheimers'] ==True)&(merge1['dementias'] ==True), 'Clinical Diagnosis'] = 'Alzheimers and Dementias'

merge1['clinical_alzheimer_diagnosis'] ='no'
merge1.loc[(merge1['Clinical Diagnosis'] =='Alzheimers and Dementias')|(merge1['Clinical Diagnosis']=='Alzheimers'),'clinical_alzheimer_diagnosis'] = 'yes'
merge1.loc[(merge1['Clinical Diagnosis'] =='Unknown'),'clinical_alzheimer_diagnosis'] = 'no'


merge1 = merge1[['participant_id', 'Year of birth PwD', 'Gender PwD','Clinical Diagnosis','clinical_alzheimer_diagnosis','Clinical Diagnosis (choice=Alzheimers Disease (typical - late onset))','Clinical Diagnosis (choice=Alzheimers Disease (atypical - early onset))',
       'Clinical Diagnosis (choice=Alzheimers Disease (atypical - posterior cortical atrophy variant))','Clinical Diagnosis (choice=Alzheimers Disease (atypical - cortico-basal syndrome))',
       'Clinical Diagnosis (choice=Alzheimers Disease (atypical - progressive aphasia variant))','Clinical Diagnosis (choice=Lewy Body Dementia)','Clinical Diagnosis (choice=Dementia in Parkinsons Disease)',
       'Clinical Diagnosis (choice=Frontotemporal dementia (unspecified))','Clinical Diagnosis (choice=Frontotemporal dementia (behavioural variant/Picks disease))',
       'Clinical Diagnosis (choice=Frontotemporal dementia (language variant/Primary Progressive Aphasia))', 'Clinical Diagnosis (choice=Vascular dementia)','Clinical Diagnosis (choice=Mild cognitive impairment (MCI))','Clinical Diagnosis (choice=Young-onset dementia)',
       'Clinical Diagnosis (choice=Alcohol-related brain damage)','Clinical Diagnosis (choice=Mixed Alzheimers and Vascular Dementia)','Clinical Diagnosis (choice=Dementia in Alzheimer disease, atypical or mixed type)','Clinical Diagnosis (choice=Normal Pressure Hydrocephalus)',
       'Clinical Diagnosis (choice=Traumatic Brain Injury)','Clinical Diagnosis (choice=No dementia)','Clinical Diagnosis (choice=Stroke)','Clinical Diagnosis (choice=Frailty)']]

#merge1.to_excel("redcap_demographics_diagnosis_nov16.xlsx")

merge1.head(5)


In [ ]:
comparison = merge1[['participant_id','Clinical Diagnosis','clinical_alzheimer_diagnosis']].merge(
    merge_research[['participant_id','Research Diagnosis','research_alzheimer_diagnosis']], on='participant_id')
comparison['its clinical and research alz diag'] = 'yes'
comparison.loc[comparison['clinical_alzheimer_diagnosis'] !=comparison['research_alzheimer_diagnosis'], 'its clinical and research alz diag']='no'
comparison

In [ ]:
comparison['Analysis Diagnosis']=comparison.apply(lambda x: x['Research Diagnosis'] if x['Research Diagnosis']!='Unknown'
                                                  else x['Clinical Diagnosis'], axis = 1)
comparison['analysis_alzheimer_diagnosis'] ='no'
comparison.loc[(comparison['Analysis Diagnosis'] =='Alzheimers and Dementias')|
               (comparison['Analysis Diagnosis']=='Alzheimers'),'analysis_alzheimer_diagnosis'] = 'yes'

comparison

In [ ]:
comparison.columns

In [ ]:
comparison[['participant_id', 'Clinical Diagnosis', 'Research Diagnosis','Analysis Diagnosis','clinical_alzheimer_diagnosis',
            'research_alzheimer_diagnosis','analysis_alzheimer_diagnosis']].to_excel("comparison_between_clinical_research_diagnosis_13_FEB_2024.xlsx")

In [ ]:
comparison[comparison['analysis_alzheimer_diagnosis']=='yes']

In [ ]:
comparison[(comparison['research_alzheimer_diagnosis']!=comparison['analysis_alzheimer_diagnosis'])]

In [ ]:
br

In [ ]:
#comparison[(comparison['Research Diagnosis']=='Unknown')|(comparison['Clinical Diagnosis']=='Unknown')]
len(comparison[(comparison['Research Diagnosis']=='Unknown')])

In [ ]:
len(comparison[(comparison['same_alzheimer_diagnosis']=='no')])

In [ ]:
len(comparison[(comparison['same_alzheimer_diagnosis']=='no')&(comparison['Research Diagnosis']=='Unknown')])

In [ ]:
len(comparison[(comparison['same_alzheimer_diagnosis']=='no')&(comparison['research_alzheimer_diagnosis']=='yes')])
comparison[(comparison['same_alzheimer_diagnosis']=='no')&(comparison['research_alzheimer_diagnosis']=='yes')]

In [ ]:
len(comparison[(comparison['same_alzheimer_diagnosis']=='no')&(comparison['research_alzheimer_diagnosis']=='no')])
comparison[(comparison['same_alzheimer_diagnosis']=='no')&(comparison['research_alzheimer_diagnosis']=='no')]

In [ ]:
len(comparison[(comparison['research_alzheimer_diagnosis']=='yes')]),len(comparison[(comparison['clinical_alzheimer_diagnosis']=='yes')])

In [ ]:
len(comparison[(comparison['research_alzheimer_diagnosis']=='no')])